In [1]:
import pandas as pd
from riotwatcher import LolWatcher, ApiError
import random

# golbal variables
# Go here to get an API key: https://developer.riotgames.com/
api_key = 
watcher = LolWatcher(api_key)
my_region = 'na1'

In [ ]:
matchID = watcher.match.matchlist_by_puuid('americas','7iUBVb7UtD2stL3K3WimGAyK1znSnxNFtjTvR_9Uz6W1Lc8sQ5DEA-wwZO2C3ZozmEjhZ7HypF2Edw')[2]
match_detail = watcher.match.by_id('americas', matchID)

participants = []
print(match_detail['info']['queueId'])
print(match_detail['info']['participants'][2]['puuid'])
for row in match_detail['info']['participants']:
    participants_row = {}
    participants_row['champion'] = row['championName']
    participants_row['summoner'] = row['summonerName']
    participants_row['teamID'] = row['teamId']
    participants_row['summoner2Id'] = row['summoner2Id']
    participants_row['win'] = row['win']
    participants_row['kills'] = row['kills']
    participants_row['deaths'] = row['deaths']
    participants_row['assists'] = row['assists']
    participants_row['totalDamageDealt'] = row['totalDamageDealt']
    participants_row['spell1Casts'] = row['spell1Casts']
    participants_row['champLevel'] = row['champLevel']
    participants_row['totalMinionsKilled'] = row['totalMinionsKilled']
    participants_row['item0'] = row['item0']
    participants_row['item1'] = row['item1']
    participants_row['CC'] = row['timeCCingOthers']
    participants_row['GIn'] = row['goldEarned']
    participants_row['GOut'] = row['goldSpent']
    participants_row['teamPosition'] = row['teamPosition']
    participants_row['puuid'] = row['puuid']
    participants.append(participants_row)
df = pd.DataFrame(participants)
df

In [2]:
# Useful Funcs

#Return puuid for given summoner name
def nameToPuuid(summonerName): # 1 request
    data = watcher.summoner.by_name(my_region, summonerName)
    puuid = data['puuid']
    return puuid

#Return matchlist for a puuid
def puuidMatchlist(puuid): # 1 request
    return watcher.match.matchlist_by_puuid('americas',puuid,count = 40)

#Take a player's most recent match and return list of teammates,opponents
def recentTeam(playerPuuid): # < 40 requests. Likely 5.
    #This variable decides if we should take the 2nd most recent game as opposed to the most.
    pick1st = False
    apiCalls = 0
    #Find most recent classic match.
    i = 0
    matchlist = puuidMatchlist(playerPuuid)
    recentMatch = matchlist[0]
    match_detail = watcher.match.by_id('americas', recentMatch)
    matchType = match_detail['info']['queueId']
    x = match_detail['info']['gameCreation']
    y = time.time()*1000
    timeAgo = (y - x)/1000
    while (matchType != 420 and matchType != 4) and i < len(matchlist) and not pick1st:
        if (matchType == 420 or matchType == 4):
            pick1st = True
        recentMatch = matchlist[i]
        match_detail = watcher.match.by_id('americas', recentMatch)
        matchType = match_detail['info']['queueId']
        i = i + 1
        #Determine how long ago this match was
        x = match_detail['info']['gameCreation']
        y = time.time()*1000
        timeAgo = (y - x)/1000
    if i == len(matchlist):
        print("No normal game found")
        return [[],[],[],len(matchlist),10000000]
    apiCalls = apiCalls + 2 + i
    
    #Create the dataframe
    participants = []
    for row in match_detail['info']['participants']:
        participants_row = {}
        participants_row['teamID'] = row['teamId']
        participants_row['puuid'] = row['puuid']
        participants.append(participants_row)
    df = pd.DataFrame(participants)
    teamA = []
    teamB = []
    
    #Determine teams
    for i in range(10):
        if df['teamID'][i] == 100:
            teamA.append(df['puuid'][i])
        else:
            teamB.append(df['puuid'][i])
    teammates = []
    opponents = []
    if playerPuuid in teamA:
        teammates = teamA
        opponents = teamB
    elif playerPuuid in teamB:
        teammates = teamB
        opponents = teamA
    else:
        print("Player not found in team.")
        return 0
    return teammates, opponents, recentMatch, apiCalls, timeAgo


In [6]:


#Compile the recent performance of a single player over their prior n games.
def indRecPerf(puuid,n): # < 40 requsts. Likely 3n.
    #Determine what matches to consider at all.
    performances = []
    matchlist = puuidMatchlist(puuid)
    gamesFound = 0
    i = 0
    while i < len(matchlist) and gamesFound < n:
        recentMatch = matchlist[i]
        match_detail = watcher.match.by_id('americas', recentMatch)
        matchType = match_detail['info']['queueId']
        if matchType == 420 or matchType == 4:
            gamesFound = gamesFound + 1
            #Analyze the match
            for j in range(10):
                if match_detail['info']['participants'][j]['puuid'] == puuid:
                    win = match_detail['info']['participants'][j]['win']
                    k = match_detail['info']['participants'][j]['kills']
                    d = match_detail['info']['participants'][j]['deaths']
                    a = match_detail['info']['participants'][j]['assists']
                    performances.append((win,k,d,a,recentMatch))
        i = i + 1
    return performances

#Only consider matches that occur BEFORE matchID.
def indRecPerfPreMatch(puuid,n,desiredMatchID):
    #Determine what matches to consider at all.
    performances = []
    foundMatchID = False
    matchlist = puuidMatchlist(puuid) # 1 call
    gamesFound = 0
    i = 0
    apiCalls = 1
    while i < len(matchlist) and gamesFound < n:
        recentMatch = matchlist[i]
        if foundMatchID:
            match_detail = watcher.match.by_id('americas', recentMatch)
            apiCalls = apiCalls + 1
            matchType = match_detail['info']['queueId']
            if matchType == 420 or matchType == 4:
                gamesFound = gamesFound + 1
                #Analyze the match
                wTotal = 0
                for j in range(len(match_detail['info']['participants'])):
                    if match_detail['info']['participants'][j]['win'] == True:
                        wTotal = wTotal + 1
                    if match_detail['info']['participants'][j]['puuid'] == puuid:
                        win = match_detail['info']['participants'][j]['win']
                        k = match_detail['info']['participants'][j]['kills']
                        d = match_detail['info']['participants'][j]['deaths']
                        a = match_detail['info']['participants'][j]['assists']
                        performances.append((win,k,d,a,recentMatch))
                if wTotal != 5: #sanity check
                    print(recentMatch)
        if matchlist[i] == desiredMatchID:
            foundMatchID = True
        i = i + 1
    return performances,apiCalls

#Compile the recent performances of teammates for their prior n games and determine if it was a win
def teamRecPerf(playerPuuid,n):
    apiCalls = 0
    allPerf = []
    teammates, opponents, recentMatchID, apiCalls, timeAgo = recentTeam(playerPuuid)
    #See if the game took place too long ago
    print("Time ago:",timeAgo)
    if timeAgo > 700000:
        return [0],0,apiCalls, 0
    if teammates == []:
        return [[],[],[],[],[]] , False, 1, 0
    for teammatePuuid in teammates:
        #print("Working on teammate...")
        indPerf,apiCalls2 = indRecPerfPreMatch(teammatePuuid,n,recentMatchID)
        allPerf.append(indPerf)
        apiCalls = apiCalls + apiCalls2 
    apiCalls = apiCalls + 1#add in the call below
    #Determine if that match was a win
    win = True
    match_detail = watcher.match.by_id('americas', recentMatchID)
    for j in range(len(match_detail['info']['participants'])):
        if match_detail['info']['participants'][j]['puuid'] == playerPuuid:
            win = match_detail['info']['participants'][j]['win']
    return allPerf, win, apiCalls, recentMatchID

In [7]:
#Randomly select a diamond player

def pickPlayer():
    div = random.randrange(1,5)
    pageNum = random.randrange(1,20)
    if div == 1:
        d1players = watcher.league.entries('na1','RANKED_SOLO_5x5','DIAMOND','I',page = pageNum)
        pglen = len(d1players)
        player2pick = random.randrange(pglen)
        player = d1players[player2pick]
        return player
    if div == 2:
        d2players = watcher.league.entries('na1','RANKED_SOLO_5x5','DIAMOND','II',page = pageNum)
        pglen = len(d2players)
        player2pick = random.randrange(pglen)
        player = d2players[player2pick]
        return player
    if div == 3:
        d3players = watcher.league.entries('na1','RANKED_SOLO_5x5','DIAMOND','III',page = pageNum)
        pglen = len(d3players)
        player2pick = random.randrange(pglen)
        player = d3players[player2pick]
        return player
    if div == 4:
        d4players = watcher.league.entries('na1','RANKED_SOLO_5x5','DIAMOND','IV',page = pageNum)
        pglen = len(d4players)
        player2pick = random.randrange(pglen)
        player = d4players[player2pick]
        return player
    return 0 #If we actually return this, there's an issue

In [ ]:
import xlwt
from xlwt import Workbook
import pandas as pd
import time

n = 3

watcher = LolWatcher(api_key)

netApiCalls = 0
#Body of program
for i in range(250):
    # Get player info
    summoner = pickPlayer()
    summonerName = summoner['summonerName']
    print("Summoner name:", summonerName)
    playerPuuid = nameToPuuid(summonerName)
    allPerf, win, apiCalls,recentMatchID = teamRecPerf(playerPuuid,n)
    if allPerf == [0]:
        netApiCalls = netApiCalls + apiCalls + 2
        print(i,apiCalls,netApiCalls)
        start_time = time.time()
        if(time.time() - start_time < 125 and netApiCalls > 45): #Time to sleep
            print("sleeping...")
            while(time.time() - start_time < 125):
                time.sleep(2)
            netApiCalls = 0
        continue
    apiCalls = apiCalls + 2 #account for the above
    if allPerf[0] != [] or allPerf[1] != [] or allPerf[2] != [] or allPerf[3] != [] or allPerf[4] != []: #Filter out times when we accidentally overload the api
        df = pd.read_excel("/Users/Tory/Documents/Documents - MacBook Pro (4)/Loldata.xlsx")
        df2 = {"P1":allPerf[0],"P2":allPerf[1],"P3":allPerf[2],"P4":allPerf[3],"P5":allPerf[4],"win":win,"match":recentMatchID}
        df = df.append(df2, ignore_index = True)
        df.to_excel("/Users/Tory/Documents/Documents - MacBook Pro (4)/Loldata.xlsx",index=False)
        print("added")
    netApiCalls = netApiCalls + apiCalls
    print(i,apiCalls,netApiCalls)
    start_time = time.time()
    if(time.time() - start_time < 125 and netApiCalls > 45): #Time to sleep
        print("sleeping...")
        while(time.time() - start_time < 125):
            time.sleep(2)
        netApiCalls = 0

Summoner name: eggbenny
Time ago: 141579.9167590332
added
0 46 46
sleeping...
Summoner name: ShiruTnT
Time ago: 442233.5381760254
added
1 25 25
Summoner name: BADˉˉˉˉˉ
Time ago: 56903.997853027344
added
2 25 50
sleeping...
Summoner name: eule
Time ago: 309720.53892700194
added
3 33 33
Summoner name: Live for the Day
Time ago: 29219.9554140625
added
4 33 66
sleeping...
Summoner name: RdySettgo
Time ago: 565023.8210183105
added
5 36 36
Summoner name: Cow Go Moooo
Time ago: 437589.5545708008
added
6 26 62
sleeping...
Summoner name: CR Ruby DAD
Time ago: 4643380.396415039
7 39 41
Summoner name: BlTES ZAH DUSTO
Time ago: 1457731.0010629883
8 2 45
Summoner name: apotheosís
Time ago: 311065.6457211914
added
9 28 73
sleeping...
Summoner name: TinyKing
Time ago: 3365.9300979003906
added
10 26 26
Summoner name: sk3leton
Time ago: 657017.2777199707
added
11 36 62
sleeping...
Summoner name: DiceRoll
Time ago: 67039.20038305664
added
12 26 26
Summoner name: EmTeed Fan
Time ago: 476906.87953295896
a

In [ ]:
#testing
matchID = watcher.match.matchlist_by_puuid('americas',nameToPuuid('ChiefDelphi'))[2]
match_detail = watcher.match.by_id('americas', matchID)

participants = []
x = match_detail['info']['gameCreation']
y = time.time()*1000
print((y - x)/1000)
print(match_detail['info']['participants'][0]['summonerName'])